# Confidence Interval of Model Parameter

A confidence interval is a range of values which is expected, with some quantifiable degree of confidence, to contain the value of an unknown value of interest.(Petty, 2012) For example, suppose a random sample of 100 boxes of cereal is selected from among all of the boxes filled by an automatic filling machine during a work shift. The mean weight of the 100 boxes in the sample is found to be 12.05 ounces and the standard deviation to be 0.1 ounces. Using the procedures to be described in the next section, we can calculate an interval [12.0304, 12.0696] for the mean weight of all boxes filled at the station and associate a confidence level of 0.95 (95%) with that interval.1 We call the calculated interval [12.0304, 12.0696], together with its associated confidence level, a confidence interval.

Here, we use a simple regression model to get the confidence interval and hopefully to complement what professor taught by the boostrapping method.